## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [5]:
import pyconll
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
import nltk
import numpy as np

In [6]:
full_train = pyconll.load_from_file(r'D:\train\pos/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file(r'D:\train\pos/ru_syntagrus-ud-dev.conllu')

In [7]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [8]:
len(full_test),len(full_train)

(6584, 48814)

In [9]:
# Приведем данные с списку списков, где слова будут кортежами
fdata_train = []
for sent in full_train:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [10]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [11]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8772537323492737

In [12]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger,verbose=True)
bigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=3165, backoff=89.74%, pruning=98.45%]


0.8829828463586425

In [13]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger,verbose=True)
trigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=4013, backoff=92.13%, pruning=98.76%]


0.882081353418933

In [14]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger,verbose=True)
trigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=5953, backoff=91.77%, pruning=98.16%]


0.881769622215482

In [15]:
bigram_tagger = BigramTagger(fdata_train, backoff=trigram_tagger,verbose=True)
bigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=1183, backoff=90.22%, pruning=99.42%]


0.8827385164964783

In [16]:
unigram_tagger = UnigramTagger(fdata_train,backoff=bigram_tagger,verbose=True)
unigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=561, backoff=83.58%, pruning=99.48%]


0.8773801098641864

In [17]:
unigram_tagger = UnigramTagger(fdata_train,backoff=trigram_tagger,verbose=True)
unigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=561, backoff=83.58%, pruning=99.48%]


0.8773801098641864

In [18]:
# Выделим отдельно списки токенов и с списки меток в трейне и тесте
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [19]:
train_tok[:5],train_label[:5]

(['Анкета', '.', 'Начальник', 'областного', 'управления'],
 ['NOUN', 'PUNCT', 'NOUN', 'ADJ', 'NOUN'])

In [20]:
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np

In [21]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [22]:
train_enc_labels,test_enc_labels,le.classes_

(array([ 7, 13,  7, ...,  9,  7, 13], dtype=int64),
 array([ 7, 13,  1, ..., 10, 16, 13], dtype=int64),
 array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
        'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
        'VERB', 'X'], dtype='<U6'))

In [23]:
countevctorizer = CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 1),dtype = np.float32 )
X_train = countevctorizer.fit_transform(train_tok)
X_test = countevctorizer.transform(test_tok)
X_train.shape,len(train_tok)

((871526, 98088), 871526)

In [24]:
%%time
lr = LogisticRegression(random_state=0,max_iter=2000,n_jobs = 15)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)

accuracy_score(test_enc_labels, pred)

Wall time: 10min 48s


0.75674855929633

In [25]:
%%time
lgbc = LGBMClassifier()
lgbc.fit(X_train, train_enc_labels)
pred = lgbc.predict(X_test)
accuracy_score(test_enc_labels, pred)


Wall time: 18.4 s


0.19120075489502242

In [26]:
vectrasers = {'Count_word_lowercase_True_(1,1)': CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 1),dtype = np.float32 ),
              'Count_word_lowercase_True_(1,2)': CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 2),dtype = np.float32 ),
              'Count_word_lowercase_True_(1,3)': CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 3),dtype = np.float32 ),
              'Count_word_lowercase_False_(1,1)': CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 1),dtype = np.float32 ),
              'Count_word_lowercase_False_(1,2)': CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 2),dtype = np.float32 ),
              'Count_word_lowercase_False_(1,3)': CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 3),dtype = np.float32 ),
              'Count_char_wb_lowercase_False_(1,4)': CountVectorizer(lowercase=False,analyzer='char_wb',ngram_range=(1, 4),dtype = np.float32 ),
              'Count_char_wb_lowercase_False_(1,6)': CountVectorizer(lowercase=False,analyzer='char_wb',ngram_range=(1, 6),dtype = np.float32 ),
              'Count_char_wb_lowercase_False_(2,10)': CountVectorizer(lowercase=False,analyzer='char_wb',ngram_range=(2, 10),dtype = np.float32 ),
              'Hash_char_(1,5)_f_500':HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000),
              'Hash_word_(1,1)_f_500':HashingVectorizer(ngram_range=(1, 1), analyzer='word', n_features=1000),
              'Hash_word_(1,2)_f_500':HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=1000),
              'Hash_word_(1,3)_f_500':HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000),
              'Hash_char_(1,3)_f_500':HashingVectorizer(ngram_range=(1, 10), analyzer='char', n_features=1000),
    
}


In [144]:
result = pd.DataFrame()

In [146]:
%%time

for vec in vectrasers:
    X_train = vectrasers[vec].fit_transform(train_tok)
    X_test = vectrasers[vec].transform(test_tok) 
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)

    l = accuracy_score(test_enc_labels, pred)
    result = result.append({'vect':vec,'model':'lr','res':l},ignore_index=True)
    
    lgbc.fit(X_train, train_enc_labels)
    
    pred = lgbc.predict(X_test)
    b = accuracy_score(test_enc_labels, pred)
    
    result = result.append({'vect':vec,'model':'lg','res':b},ignore_index=True)
    
    
    

Wall time: 3h 29min 40s


In [149]:
result.sort_values('res',ascending=False)

,vect,model,res
14,"Count_char_wb_lowercase_False_(1,6)",lr,0.963300
16,"Count_char_wb_lowercase_False_(2,10)",lr,0.963098
12,"Count_char_wb_lowercase_False_(1,4)",lr,0.962407
18,"Hash_char_(1,5)_f_500",lr,0.888122
26,"Hash_char_(1,3)_f_500",lr,0.871702
27,"Hash_char_(1,3)_f_500",lg,0.811259
4,"Count_word_lowercase_True_(1,3)",lr,0.757507
2,"Count_word_lowercase_True_(1,2)",lr,0.757507
0,"Count_word_lowercase_True_(1,1)",lr,0.756749
10,"Count_word_lowercase_False_(1,3)",lr,0.750329


Как можно заметить прекрасные результаты дает логистическая регрессия на коунт векторах по буквенно (1,6),(2,10),(1,4). В пределах слова.  
Бустинг вообще работает только с хэш ветором. В любом случае analyzer='word', хуже чем char


# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [28]:
import corus
from corus import load_ne5
from razdel import tokenize
import pandas as pd

In [29]:
patch_coll_5 = r'D:\train\pos\collection5/'

In [30]:
records = load_ne5(patch_coll_5)
next(records)

Ne5Markup(
    id='001',
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[Ne5Span(
         index='T1',
         type='GEOPOLIT',
         start=0,
         st

процедуры обработки взять из вебинарного ноутбука

In [31]:
words_docs = []
docs_words=[]
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    docs_words.append([rec.text,words]) 
    words_docs.extend(words)

In [32]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_docs = pd.DataFrame(docs_words, columns=['text', 'tag'])

In [34]:
df_docs

,text,tag
0,Комиссар СЕ критикует ограничительную политику...,"[[Комиссар, OUT], [СЕ, ORG], [критикует, OUT],..."
1,"Пулеметы, автоматы и снайперские винтовки изъя...","[[Пулеметы, OUT], [,, OUT], [автоматы, OUT], [..."
2,4 октября назначены очередные выборы Верховног...,"[[4, OUT], [октября, OUT], [назначены, OUT], [..."
3,Следственное управление при прокуратуре требуе...,"[[Следственное, ORG], [управление, ORG], [при,..."
4,В Нижегородской области осудили бывшего началь...,"[[В, OUT], [Нижегородской, LOC], [области, LOC..."
...,...,...
994,"Депутат от ""ЕР"": К отставке А.Сердюкова причас...","[[Депутат, OUT], [от, OUT], ["", ORG], [ЕР, ORG..."
995,\r\nСи Цзиньпин избран генсеком Коммунистическ...,"[[Си, PER], [Цзиньпин, PER], [избран, OUT], [г..."
996,"""Ведомости"" узнали о смене лидера московских е...","[["", MEDIA], [Ведомости, MEDIA], ["", MEDIA], [..."
997,СМИ узнали о кутежах туркменского чиновника на...,"[[СМИ, MEDIA], [узнали, OUT], [о, OUT], [кутеж..."


In [35]:
df_words['tag'].value_counts()

OUT         219112
PER          21196
ORG          13650
LOC           4567
GEOPOLIT      4342
MEDIA         2480
Name: tag, dtype: int64

In [36]:
def show_ner_text(n):
    print("TEXT")
    print()
    print(df_docs.text.loc[n])
    print()
    print('NER')  
    print()
    print([i for i in df_docs.tag.loc[n] if i[1]!='OUT'])
    l = {i[1] for i in df_docs.tag.loc[n]}-{"OUT"}
    print()
    print('wich one ner')      
    print(l)
    print(len(l))

In [37]:
show_ner_text(0)

TEXT

Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах

05/08/2008 10:32

МОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.

"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.

По сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".

Комиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, оказались разделенными".



In [38]:
def ner_nltk(n):
    print("TEXT")
    text = df_docs.text.loc[n]
    print()
    print(text)
    print()
    tokens = nltk.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    ner = {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(pos) if hasattr(chunk, 'label') }
    print('NER')  
    print()
    print(ner)
    l = {i[1] for i in df_docs.tag.loc[n]}-{"OUT"}
    print()
#     print('wich one ner')      
#     print(l)
#     print(len(l))
    

In [39]:
ner_nltk(0)

TEXT

Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах

05/08/2008 10:32

МОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.

"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.

По сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".

Комиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, оказались разделенными".



In [173]:
from sklearn import model_selection, preprocessing, linear_model

train_x, test_x, train_y , test_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [41]:
test_y

array([4, 4, 4, ..., 4, 4, 4])

In [42]:
train_y

array([4, 4, 3, ..., 4, 4, 4])

In [43]:
encoder.classes_

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype=object)

In [44]:
train_x,test_x

(188198         штате
 102970         уходу
 219144             e
 95130      освободил
 259378           Она
              ...    
 222468         сумму
 89881          своем
 192373    президенты
 149982             и
 117937        активы
 Name: word, Length: 199010, dtype: object,
 201232              -
 88375               ,
 255007             их
 179781         Джобса
 9760      экстремизма
              ...     
 63391               в
 135935       Напомним
 209141              .
 176269         личных
 13846       принимала
 Name: word, Length: 66337, dtype: object)

In [92]:
result_ner = pd.DataFrame()

In [134]:
%%time

for vec in vectrasers:
    X_train = vectrasers[vec].fit_transform(train_x)
    X_test = vectrasers[vec].transform(test_x) 
    lr.fit(X_train, train_y)
    pred = lr.predict(X_test)

    l = accuracy_score(test_y, pred)
    result_ner = result_ner.append({'vect':vec,'model':'lr','res':l},ignore_index=True)
    print(result_ner)
    
    lgbc.fit(X_train, train_y)
    
    pred = lgbc.predict(X_test)
    b = accuracy_score(test_y, pred)
    
    result_ner = result_ner.append({'vect':vec,'model':'lg','res':b},ignore_index=True)
    print(result_ner)

                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
2  Count_word_lowercase_True_(1,1)    lg  0.872801
                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
2  Count_word_lowercase_True_(1,1)    lg  0.872801
3  Count_word_lowercase_True_(1,2)    lr  0.910548
                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
2  Count_word_lowercase_True_(1,1)    lg  0.872801
3  Count_word_lowercase_True_(1,2)    lr  0.910548
4  Count_word_lowercase_True_(1,2)    lg  0.872801
                              vect model       res
0  Count_word_lowercase_True_(1

                                    vect model       res
0        Count_word_lowercase_True_(1,1)    lr  0.910337
1        Count_word_lowercase_True_(1,1)    lr  0.910337
2        Count_word_lowercase_True_(1,1)    lg  0.872801
3        Count_word_lowercase_True_(1,2)    lr  0.910548
4        Count_word_lowercase_True_(1,2)    lg  0.872801
5        Count_word_lowercase_True_(1,3)    lr  0.910548
6        Count_word_lowercase_True_(1,3)    lg  0.872801
7       Count_word_lowercase_False_(1,1)    lr  0.914166
8       Count_word_lowercase_False_(1,1)    lg  0.873992
9       Count_word_lowercase_False_(1,2)    lr  0.914316
10      Count_word_lowercase_False_(1,2)    lg  0.873992
11      Count_word_lowercase_False_(1,3)    lr  0.914271
12      Count_word_lowercase_False_(1,3)    lg  0.873992
13   Count_char_wb_lowercase_False_(1,4)    lr  0.956193
14   Count_char_wb_lowercase_False_(1,4)    lg  0.948777
15   Count_char_wb_lowercase_False_(1,6)    lr  0.956872
16   Count_char_wb_lowercase_Fa

                                    vect model       res
0        Count_word_lowercase_True_(1,1)    lr  0.910337
1        Count_word_lowercase_True_(1,1)    lr  0.910337
2        Count_word_lowercase_True_(1,1)    lg  0.872801
3        Count_word_lowercase_True_(1,2)    lr  0.910548
4        Count_word_lowercase_True_(1,2)    lg  0.872801
5        Count_word_lowercase_True_(1,3)    lr  0.910548
6        Count_word_lowercase_True_(1,3)    lg  0.872801
7       Count_word_lowercase_False_(1,1)    lr  0.914166
8       Count_word_lowercase_False_(1,1)    lg  0.873992
9       Count_word_lowercase_False_(1,2)    lr  0.914316
10      Count_word_lowercase_False_(1,2)    lg  0.873992
11      Count_word_lowercase_False_(1,3)    lr  0.914271
12      Count_word_lowercase_False_(1,3)    lg  0.873992
13   Count_char_wb_lowercase_False_(1,4)    lr  0.956193
14   Count_char_wb_lowercase_False_(1,4)    lg  0.948777
15   Count_char_wb_lowercase_False_(1,6)    lr  0.956872
16   Count_char_wb_lowercase_Fa

In [135]:
result_ner.sort_values('res',ascending=False)

,vect,model,res
17,"Count_char_wb_lowercase_False_(2,10)",lr,0.956947
15,"Count_char_wb_lowercase_False_(1,6)",lr,0.956872
13,"Count_char_wb_lowercase_False_(1,4)",lr,0.956193
16,"Count_char_wb_lowercase_False_(1,6)",lg,0.949048
14,"Count_char_wb_lowercase_False_(1,4)",lg,0.948777
18,"Count_char_wb_lowercase_False_(2,10)",lg,0.947269
20,"Hash_char_(1,5)_f_500",lg,0.931139
28,"Hash_char_(1,3)_f_500",lg,0.930145
9,"Count_word_lowercase_False_(1,2)",lr,0.914316
11,"Count_word_lowercase_False_(1,3)",lr,0.914271


In [307]:
import tensorflow as tf

from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, \
GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input,BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.activations import relu,sigmoid,tanh

In [222]:
df_words

,word,tag
0,Комиссар,OUT
1,СЕ,ORG
2,критикует,OUT
3,ограничительную,OUT
4,политику,OUT
...,...,...
265342,замглавы,OUT
265343,Бердска,LOC
265344,Владимир,PER
265345,Штоп,PER


In [266]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [344]:
train_x, test_x, train_y , test_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

In [346]:
vec = CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 1),dtype = np.float32 )

In [347]:
X_train = vec.fit_transform(train_x)
X_test = vec.transform(test_x)
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)
train_y = tf.one_hot(train_y,depth=6)
test_y = tf.one_hot(test_y,depth=6)

In [348]:
X_test.shape,X_train.shape

((66337, 29260), (199010, 29260))

In [349]:
X_train_ = convert_sparse_matrix_to_sparse_tensor(X_train)
X_train_ = tf.sparse.reorder(X_train_)
X_test_ = convert_sparse_matrix_to_sparse_tensor(X_test)
X_test_ = tf.sparse.reorder(X_test_)

In [354]:
x_input = Input(shape=(29260))

x = Dense(64)(x_input)
x = BatchNormalization()(x)
x = sigmoid(x)
x = Dense(32)(x)
x = BatchNormalization()(x)
x = tanh(x)
x = Dense(16)(x)
x = BatchNormalization()(x)
x = relu(x)
output = Dense(6,activation='softmax')(x)

model =Model(x_input,output)


In [355]:
model.compile(optimizer='Adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [356]:
hh = model.fit(X_train_,train_y,epochs=10,batch_size = 2048,validation_data=(X_test_, test_y),verbose=1)
model.evaluate(X_test_, test_y)

Epoch 1/10


C:\Users\lqw\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_22/dense_85/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_22/dense_85/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model_22/dense_85/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


98/98 [==============================] - 2s 14ms/step - loss: 0.8367 - accuracy: 0.8262 - val_loss: 1.1010 - val_accuracy: 0.8282
Epoch 2/10
98/98 [==============================] - 1s 13ms/step - loss: 0.3087 - accuracy: 0.9516 - val_loss: 1.0424 - val_accuracy: 0.8282
Epoch 3/10
98/98 [==============================] - 1s 13ms/step - loss: 0.2114 - accuracy: 0.9578 - val_loss: 1.0058 - val_accuracy: 0.8778
Epoch 4/10
98/98 [==============================] - 1s 13ms/step - loss: 0.1778 - accuracy: 0.9599 - val_loss: 0.7232 - val_accuracy: 0.9276
Epoch 5/10
98/98 [==============================] - 1s 13ms/step - loss: 0.1622 - accuracy: 0.9605 - val_loss: 0.3881 - val_accuracy: 0.9369
Epoch 6/10
98/98 [==============================] - 1s 13ms/step - loss: 0.1526 - accuracy: 0.9610 - val_loss: 0.2790 - val_accuracy: 0.9386
Epoch 7/10
98/98 [==============================] - 1s 13ms/step - loss: 0.1476 - accuracy: 0.9611 - val_loss: 0.2444 - val_accuracy: 0.9392
Epoch 8/10
98/98 [======

[0.23959730565547943, 0.9391440749168396]

In [326]:
df_words

,word,tag
0,Комиссар,OUT
1,СЕ,ORG
2,критикует,OUT
3,ограничительную,OUT
4,политику,OUT
...,...,...
265342,замглавы,OUT
265343,Бердска,LOC
265344,Владимир,PER
265345,Штоп,PER


Добавим слова до и после ключевого

In [359]:
words = []

key_word = '.'
befor_word = "."
after_word = '.'

for ind,i in df_words.word.items():
    if i =='.':
        words.append('.')
        continue
    key_word = i
    if ind>1 :
        befor_word = df_words.word.loc[ind-1]
    else:
        befor_word = "." 
    if ind<len(df_words) :
        after_word = df_words.word.loc[ind+1]
    else:
        after_word = "."    
        
    words.append(', '.join([befor_word,key_word,after_word]))
    


In [360]:
df_words['words'] = words
df_words

,word,tag,words
0,Комиссар,OUT,"., Комиссар, СЕ"
1,СЕ,ORG,"., СЕ, критикует"
2,критикует,OUT,"СЕ, критикует, ограничительную"
3,ограничительную,OUT,"критикует, ограничительную, политику"
4,политику,OUT,"ограничительную, политику, в"
...,...,...,...
265342,замглавы,OUT,"первый, замглавы, Бердска"
265343,Бердска,LOC,"замглавы, Бердска, Владимир"
265344,Владимир,PER,"Бердска, Владимир, Штоп"
265345,Штоп,PER,"Владимир, Штоп, ."


In [361]:
train_x, test_x, train_y , test_y = model_selection.train_test_split(df_words['words'], df_words['tag'])
vec = CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 1),dtype = np.float32 )
X_train = vec.fit_transform(train_x)
X_test = vec.transform(test_x)
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)
train_y = tf.one_hot(train_y,depth=6)
test_y = tf.one_hot(test_y,depth=6)
X_train_ = convert_sparse_matrix_to_sparse_tensor(X_train)
X_train_ = tf.sparse.reorder(X_train_)
X_test_ = convert_sparse_matrix_to_sparse_tensor(X_test)
X_test_ = tf.sparse.reorder(X_test_)
X_test_.shape,X_train_.shape


(TensorShape([66337, 33705]), TensorShape([199010, 33705]))

In [365]:
x_input = Input(shape=(33705))

x = Dense(64)(x_input)
x = BatchNormalization()(x)
x = sigmoid(x)
x = Dense(32)(x)
x = BatchNormalization()(x)
x = tanh(x)
x = Dense(16)(x)
x = BatchNormalization()(x)
x = relu(x)
output = Dense(6,activation='softmax')(x)

model =Model(x_input,output)

model.compile(optimizer='Adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

hh_1 = model.fit(X_train_,train_y,epochs=20,batch_size = 2048,validation_data=(X_test_, test_y),verbose=1)
model.evaluate(X_test_, test_y)


Epoch 1/20


C:\Users\lqw\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_26/dense_101/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_26/dense_101/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model_26/dense_101/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


98/98 [==============================] - 2s 15ms/step - loss: 0.7730 - accuracy: 0.8265 - val_loss: 1.0085 - val_accuracy: 0.8246
Epoch 2/20
98/98 [==============================] - 2s 14ms/step - loss: 0.3311 - accuracy: 0.9158 - val_loss: 0.7785 - val_accuracy: 0.8246
Epoch 3/20
98/98 [==============================] - 2s 14ms/step - loss: 0.2220 - accuracy: 0.9328 - val_loss: 0.6021 - val_accuracy: 0.8255
Epoch 4/20
98/98 [==============================] - 2s 14ms/step - loss: 0.1756 - accuracy: 0.9434 - val_loss: 0.4201 - val_accuracy: 0.8806
Epoch 5/20
98/98 [==============================] - 2s 14ms/step - loss: 0.1480 - accuracy: 0.9512 - val_loss: 0.3297 - val_accuracy: 0.8881
Epoch 6/20
98/98 [==============================] - 2s 14ms/step - loss: 0.1290 - accuracy: 0.9569 - val_loss: 0.2941 - val_accuracy: 0.8995
Epoch 7/20
98/98 [==============================] - 2s 14ms/step - loss: 0.1151 - accuracy: 0.9613 - val_loss: 0.3025 - val_accuracy: 0.8993
Epoch 8/20
98/98 [======

[0.3945581018924713, 0.8952168226242065]

В целом простая сеть хуже решает задачу. Добовление смежных слов , ухудштло показатели